<div style="text-align: center;">
    <img src="./images/logo_CESI.jpg" width="500">
</div>

# Livrable 1

## Contexte du Projet

Ce projet vise à répondre à l'appel à manifestation d'intérêt lancé par l'ADEME (Agence de l'Environnement et de la Maîtrise de l'Énergie) pour la réalisation de démonstrateurs et d'expérimentations de nouvelles solutions de mobilité durable adaptées à différents types de territoires.

L'équipe de CesiCDP, en collaboration avec plusieurs partenaires, s'est déjà intéressée à la mobilité multimodale intelligente et souhaite développer une méthode basée sur la recherche opérationnelle pour résoudre le problème de la gestion de tournées de livraison. L'objectif est de calculer une tournée optimisée sur un réseau routier reliant un sous-ensemble de villes, en minimisant la durée totale de la tournée tout en tenant compte du trafic prévu sur chaque axe pour les différentes tranches horaires.

Le projet comporte une version de base du problème où le modèle et le code en Python doivent être développés pour résoudre des instances de taille importante. De plus, une étude statistique du comportement expérimental de l'algorithme doit être réalisée.

Dans un second temps, des contraintes supplémentaires peuvent être intégrées. Ces dernières incluent des fenêtres de temps de livraison, l'interdiction de livrer en dehors de ces fenêtres, la possibilité d'attendre sur place l'ouverture des fenêtres, l'utilisation de plusieurs camions pour effectuer les livraisons avec des contraintes de capacité et d'encombrement, des points de collecte spécifiques pour chaque objet, la variation du temps de parcours des axes en fonction du trafic, etc.

Le projet est organisé en plusieurs étapes, dont la modélisation formelle, la conception algorithmique et l'implémentation, l'étude expérimentale et la présentation des résultats à l'équipe avant la remise des livrables à l'ADEME.

Le premier livrable de modélisation contient une étude décrivant le problème, sa formalisation, les contraintes supplémentaires traitées et une analyse théorique de sa complexité.

## Membre du groupe

<div style="text-align: center;">
<img src="./images/team.png" width="1000">
</div>

# Optimisation de Tournées de Livraison

## Choix des contraintes et explication  

##### Fenêtre de temps de livraison pour chaque objet
- Interdiction de livrer hors de la fenêtre
- Possibilité d'attendre sur place l'ouverture de la fenêtre temporelle
##### k camions disponibles simultanément pour effectuer les livraisons. Le calcul de la tournée devra inclure l’affectation des objets (et donc des points de livraison) aux différents camions disponibles, et minimiser non plus le temps total, mais la date de retour du dernier camion à la base. 
- Capacité des camions (2 ou 3 dimensions) et encombrement des objets 
- Certains objets ne peuvent être livrés que par certains camions

## Plan d'action 

- Reformulation du contexte 
- Calcul de propriété théorique 
- Calcul de complexité 
- Représentation formelle des données et de l'objectif (Contraintes) 
- Sources

## Contexte
L'ADEME a lancé un appel à manifestation d'intérêt pour promouvoir la réalisation de démonstrateurs et d’expérimentations de nouvelles solutions de mobilité. CesiCDP, en collaboration avec ses partenaires, propose une solution pour optimiser les tournées de livraison.

## Description du Problème de Base
Le problème consiste à calculer une tournée de livraison sur un réseau routier reliant plusieurs villes et revenant au point de départ, en minimisant la durée totale de la tournée. Ce problème est connu sous le nom de Problème du Voyageur de Commerce (TSP).


# Vocabulaire et notion de complexité

## Définition de base

Un graphe est composé de sommets reliés par des arêtes.
Une boucle est une arête reliant un sommet à lui-même.
Deux sommets reliés par une arête sont dit adjacents 
Le degré d'un sommet est le nombre d'arêtes dont ce sommet est une extrémité.
Un sommet qui n'est adjacent à aucun autre sommet du graphe est dit isolé. C'est le cas du sommet E de notre exemple. 

<div style="text-align: center;">
<img src="./images/graph_def.png">
</div>

Un graphe orienté est un graphe dont les arêtes sont orientées. Chaque arête ne peut être parcourue que dans le sens de la flèche.
Les arêtes sont appelées arcs.

<div style="text-align: center;">
<img src="./images/graph_oriente.png">
</div>

Un graphe simple est un graphe ayant au plus une arête entre deux sommets et n’ayant pas de boucle.
Un graphe est dit complet lorsque tous ses sommets sont adjacents 


<div style="text-align: center;">
<img src="./images/graph_comp_non_comp.png">
</div>

- La somme des degrés des sommets d’un graphe non orienté est égale au double du nombre total d’arêtes.

## Chaines 

Une chaîne est une suite de sommets telle que chaque sommet est relié au suivant par une arête.
Dans une chaîne, on peut prendre plusieurs fois la même arête.
La longueur d'une chaîne est le nombre d'arêtes qui la composent.
Une chaîne est fermée lorsque l’origine et l’extrémité sont confondues.

<div style="text-align: center;">
<img src="./images/def_chaine.png">
</div>

## Cycle 

Un cycle est une chaîne  fermée (c'est-à-dire dont l'origine et l'extrémité sont identiques) dont toutes les arêtes sont distinctes.
On dit qu'un graphe est connexe si deux sommets quelconques peuvent être reliés par une chaîne.

### Cycles Eulérien 

Une chaîne est eulérienne lorsqu’elle contient chaque arête du graphe une et une seule fois.
Si cette chaîne est un cycle, il s’agit d’un cycle eulérien
Un graphe connexe contient une chaîne eulérienne si et seulement s'il possède 0 ou 2 sommets de degré impair

## Matrice d'adjacence 

### Graphe non orienté 
La matrice d’adjacence associée à un graphe non orienté d’ordre n dont les sommets sont numérotés de 1 à $n$ est la matrice carrée d’ordre $n$, où le terme situé en ligne $i$ et colonne $j$ est égal au nombre d’arêtes reliant $i$ et $j$.

La matrice d’adjacence d’un graphe non orienté est toujours symétrique par rapport à sa première diagonale.


<div style="text-align: center;">
<img src="./images/Matrice_adj_non_oriente.png">
</div>


### Graphe orienté

La matrice d’adjacence associée à un graphe orienté d’ordre $n$ dont les sommets sont numérotés de 1 à $n$ est la matrice carrée d’ordre $n$, où le terme situé en ligne $i$ et colonne $j$ est égal à 1 s’il existe une arête menant de $i$ à $j$, et à 0 sinon. Contrairement à la matrice d’adjacence d’un graphe non orienté, celle d’un graphe orienté n’est pas forcément symétrique par rapport à sa première diagonale.

<div style="text-align: center;">
<img src="./images/Matrice_adj_oriente.png">
</div>

## Complexité d'un algorithme
La complexité d'un algorithme peut être définie comme la quantité de ressources, telles que le temps d'exécution et l'espace mémoire, utilisées par cet algorithme.

- $ O(1) $ (constante): Toutes les opérations sont simples.
- $ O(log n) $: Ce sont des algorithmes très rapides. Ex : recherche dichotomique.
- $ O(n) $ (on dit linéaire): Typiquement quand on parcourt un tableau ou une liste un nombre borné de fois : recherche dans un tableau, minimum d’une liste, etc.
- $O(n log n)$: Cette complexité apparaît régulièrement lorsque l’on fait du “diviser pour régner”. Ex : tri rapide, tri fusion, tri par tas, etc.
- $O(n^2)$ (on dit quadratique). Quand on manipule des tableaux à deux dimensions, ou qu’on effectue un assez grand nombre de calculs sur un tableau à une dimension : somme de deux matrices, tri insertion, tri bulle, tri sélection, etc.


La notion de "grand O" est utilisée pour caractériser cette complexité. Elle fournit une fonction qui limite asymptotiquement, avec un facteur constant, la fonction qui représente le temps de calcul de l'algorithme en fonction de la taille de l'entrée. Par exemple, si l'on dit que l'algorithme a une complexité de $O(n²)$ dans le pire des cas, cela signifie que l'algorithme prendra au maximum un temps de l'ordre de $x * n² + y$, où x est une constante réelle. La valeur de y peut être une expression polynomiale de degré inférieur à $n²$.



## Reformulation Formelle du Problème
### Variables de Décision
- \($ x_{ij} $\): Indicateur binaire (1 si l'arc de la ville \( $i$ \) à la ville \( $j$ \) est utilisé, 0 sinon)
- \($ t_i $\): Temps d'arrivée à la ville \( $i$ \)

### Fonction Objectif
Minimiser la durée totale de la tournée:
\[
$\min \sum_{i=1}^{n} \sum_{j=1}^{n} c_{ij} x_{ij}$
\]
où \( $c_{ij}$ \) est le coût (temps de trajet) entre les villes \( i \) et \( j \).

### Contraintes
1. Chaque ville doit être visitée une fois:
\[
$\sum_{j=1}^{n} x_{ij} = 1 \quad \forall i$
\]
2. Chaque ville doit être quittée une fois:
\[
$\sum_{i=1}^{n} x_{ji} = 1 \quad \forall j$
\]
3. Temps de trajet:
\[
$t_i + c_{ij} - t_j \leq M (1 - x_{ij}) \quad \forall i, j$
\]
4. Variables binaires:
\[
$x_{ij} \in \{0, 1\}$
\]


# Description des contraintes

## Contraintes de base

<div style="text-align: center;">
  <img src="./images/OIJ_graph.drawio.png "/>
</div> 

On cherche à calculer la distance minimale entre les villes, afin de minimiser le temps de trajet. On cherche donc à minimiser la fonction suivante :
$$ Coût(total) = \sum temps \ de \ parcours \ (segment) $$
$$ \sum_{i}^{}\sum_{j}^{}x_{ij}\times c_{ij} $$

$$ F(S) = \sum_{i=1}^{n-1} \text{dist}((x_{S_i}, y_{S_i}), (x_{S_{i+1}}, y_{S_{i+1}})) + \text{dist}((x_{S_n}, y_{S_n}), (x_{S_1}, y_{S_1})) $$

Dans cette expression, $S$ est une solution candidate représentée par une séquence ordonnée des indices des points d'intérêt à visiter, $ n $ est le nombre de points d'intérêt, $(x_i, y_i)$ sont les coordonnées cartésiennes du point d'intérêt d'indice $i$, et $\text{dist}((x_i, y_i), (x_j, y_j))$ représente la distance euclidienne entre les points $(x_i, y_i)$ et $(x_j, y_j)$.

Afin de calculer le temps de trajet le plus efficace, nous pouvons utiliser l'algorithme de Dijkstra. Cet algorithme permet de calculer le plus court chemin entre deux sommets d'un graphe. Il est basé sur le principe de relaxation, qui consiste à améliorer progressivement une estimation de la distance d'un sommet à la source. L'algorithme de Dijkstra est un algorithme glouton, c'est-à-dire qu'il choisit à chaque étape le sommet qui a la distance la plus faible parmi les sommets non visités. Il est nécessaire de parcourir tous les sommets du graphe pour trouver le plus court chemin entre deux sommets.
Cependant, le rapport entre le temps d'exécution et la taille du graphe est de $O(n²)$, ce qui est trop long pour des graphes de grande taille. Il est donc nécessaire d'utiliser d'autres manières de trouver cet optimal : les heuristiques.

### On ne peut livrer le client 2 avant de livrer le client 1 

<div style="text-align: center;">
  <img src="./images/OIJ_graph.drawio.png" />
</div>



Nous allons poser notre point de départ au point O. Il s'agit tout d'abord de s'assurer que le camion part du dépôt. On pose donc un booléen pour savoir si le camion s'est déplacé. La valeur est de 0 si le camion n'a pas parcouru l'arête et 1 si l'arrête est parcourue. 

Soit : $ \sum^{}x_{oi} =1 $
et  $ \sum^{}x_{oj} =1 $



$t_i + c_{ij} - N(1 - x_{ij}) \leq t_j$

Avec $t_i$ le temps de passage au sommet $i$, $c_{ij}$ le temps de parcours entre les sommets $i$ et $j$, $N$ une constante suffisamment grande et $x_{ij}$ un booléen valant 1 si le sommet $j$ est visité après le sommet $i$ et 0 sinon.
Cette équation permet de s'assurer que le sommet $j$ n'est visité qu'après le sommet $i$. En effet, si $x_{ij}$ vaut 0, alors $N(1 - x_{ij})$ vaut $N$ et l'équation devient $t_i + c_{ij} - N \leq t_j$. Comme N est une constante suffisamment grande, cette équation est toujours vérifiée. Si $x_{ij}$ vaut 1, alors $N(1 - x_{ij})$ vaut 0 et l'équation devient $t_i + c_{ij} \leq t_j$, ce qui est équivalent à $t_i + c_{ij} - t_j \leq 0$. Cette équation est vérifiée si le sommet $j$ est visité après le sommet $i$.

### Puisque nous sommes dans un cycle hamiltonien, notre problème ne peut pas contenir de boucle :

En posant la même base que le problème précédent, on sait que nous ne pouvons passer deux fois au même endroit de sorte que : 

<div style="text-align: center;">
  <img src="./images/Not_boucle.png" />
</div>

De ce fait, on pose : $ \sum^{}x_{ii} = 0 $ 




## Contraintes Supplémentaires
#### Fenêtres de Temps de Livraison
- Interdiction de livrer hors de la fenêtre
- Possibilité d'attendre sur place

#### k camions disponibles simultanément pour effectuer les livraisons. Le calcul de la tournée devra inclure l’affectation des objets (et donc des points de livraison) aux différents camions disponibles, et minimiser non plus le temps total, mais la date de retour du dernier camion à la base.
- Capacité des camions (2 ou 3 dimensions) et encombrement des objets
- Certains objets ne peuvent être livrés que par certains camions



## Analyse supplémentaire des contraintes non implémentée

- 

- 

## Propriétés Théoriques du Problème
Le problème de base est un problème NP-difficile, similaire au Problème du Voyageur de Commerce (TSP). La complexité théorique du problème est confirmée par sa réduction au TSP, ce qui indique qu'il n'existe pas d'algorithme en temps polynomial pour le résoudre exactement.

### Comparaison avec d'autres Problèmes
- **TSP**: Chaque ville est visitée une fois avec un retour au point de départ.
- **VRP**: Similaire au TSP mais avec des véhicules multiples et des contraintes de capacité.
